# 搭建一个高可用的hdfs集群


架构图 
![architecture](1.png)




## 搭建步骤
3台机器，和一个zookeeper集群(每台机器都启动了一个zookeeper服务).

分别为master, node1, node2， 搭建想法是将master设为active namenode， node2 作为standby namenode, node1 为datanode

***
1. 下载[hadoop](http://apache.claz.org/hadoop/common/hadoop-2.9.2/hadoop-2.9.2.tar.gz)
2. 解压， 我解压的路径是/opt/bigdata/hadoop
3. 申明JAVA_HOME环境变量
4. 3台机器使用相同的配置, [hdfs-site.xml](hdfs-site.xml), [core-site.xml](core-site.xml)


- 启动JournalNode服务. (每台机器都要执行)    
    
        hadoop-daemon.sh start journalnode
        
- namenode格式化   (master, node2执行即可)
        
        hdfs namenode -format
- 启动namenode  （master)
    
        hadoop-daemon.sh start namenode
        
- 启动standby namenode (node2)

        hdfs namenode -bootstrapStandby
        hadoop-daemon.sh start namenode
        
- 在zookeeper中创建一个znode用来保存automatic failover的数据 (master, node2)
    
        hdfs zkfc –formatZK
        
- 启动DFSZKFailoverController服务
        
        hadoop-daemon.sh start zkfc
        
- 启动datanode (node1)
        
         hadoop-daemon.sh start datanode
         
- 使用cli查看namenode状态
    
        hdfs haadmin -getServiceState nn1
        hdfs haadmin -getServiceState nn2
         
更详细的说明请参考

[官方文档](https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-hdfs/HDFSHighAvailabilityWithQJM.html)

[How to Set Up Hadoop Cluster with HDFS High Availability](https://www.edureka.co/blog/how-to-set-up-hadoop-cluster-with-hdfs-high-availability/)
***

添加新的节点步骤 
- 复制所有的配置文件到新的节点
- hadoop-daemon.sh start journalnode
- hadoop-daemon.sh start datanode

        

## 测试

异常测试:  杀死master的namenode, 观察变化

1. 首先进入master的ui界面(http://master:50070)
    ![master](master.png)
    
   node2的ui界面(http://node2:50070)
   ![node2](node2.png)
2. 在master上面输入jps 获取namenode 的进程号 并杀死, 重新启动master的namenode服务, 观察ui变化

    ![master1](master-1.png)
    
    node2：
    ![node21](node2-1.png)

    

## yarn 

- 配置[yarn-site.xml](yarn-site.xml)
- 修改资源节点 修改slaves文件
- 启动sbin/start-yarn.sh

## 其他的发现

**一个高可用的架构可以在不关闭服务情况下修改配置.**

之前因为新增加节点遇到了

```
Datanode denied communication with namenode because hostname cannot be resolved 
```

google后需要添加一条配置，先关闭nn1, 然后启动启动namenode服务, 再关闭nn2，启动namenode， 这样就在不关闭服务情况下修改配置.